# Data Science Academy
# Mini-Projeto 3
# Análise Estatística de Dados com NumPy Para a Área de Marketing

## 1. Definição do Problema de Negócio

### 1.1. Contexto

Uma plataforma de e-commerce coleta um volume significativo de dados sobre a interação dos usuários com o site, incluindo o número de visitas, a duração da sessão, a atividade de adição de produtos ao carrinho e os valores de compra finalizados. No entanto, esses dados estão sendo subutilizados. Atualmente, as decisões sobre campanhas de marketing, promoções e melhorias na experiência do usuário (UX) são tomadas com base em intuição e métricas de alto nível, sem uma compreensão aprofundada dos padrões de comportamento que impulsionam os resultados.

### 1.2. Problema de Negócio

A empresa enfrenta o desafio de compreender profundamente os padrões de comportamento que diferenciam os clientes de alto valor dos visitantes que abandonam o site sem comprar. Essa falta de clareza resulta em:

- Marketing Genérico: Nossas campanhas de marketing são de "tamanho único", resultando em baixo engajamento e desperdício de orçamento, pois não conseguimos personalizar as ofertas para os segmentos de clientes corretos.

- Perda de Oportunidades: Não conseguimos identificar e engajar proativamente os clientes com maior potencial de compra ou criar estratégias para converter os visitantes que demonstram interesse, mas não finalizam a compra.

- Decisões Não Embasadas: As estratégias de produto e de experiência do usuário carecem de uma base quantitativa sólida sobre quais comportamentos (ex: tempo no site, frequência de visitas) estão mais fortemente correlacionados com o sucesso das vendas.

### 1.3. Objetivo Principal

Utilizar a análise estatística dos dados de navegação e compra para segmentar clientes, identificar os principais indicadores de comportamento que levam à conversão e fornecer insights acionáveis para as equipes de marketing e produto, a fim de aumentar o ticket médio e a taxa de conversão geral da plataforma.

### 1.4. Perguntas-Chave a Serem Respondidas

A análise de dados deve responder às seguintes perguntas críticas de negócio:

- 1- Qual é o perfil médio do nosso usuário em termos de visitas, tempo de navegação e valor de compra (ticket médio)?

- 2- Quais são as características e comportamentos distintos dos nossos clientes de "Alto Valor"? Eles visitam mais o site? Passam mais tempo navegando?

- 3- Qual é o comportamento dos usuários que visitam o site, mas não realizam nenhuma compra? Onde está a oportunidade de conversão com este grupo?

- 4- Existe uma correlação estatisticamente relevante entre o tempo gasto no site, o número de itens no carrinho e o valor final da compra?

### 1.5. Resultado Esperado e Impacto no Negócio

O resultado deste projeto será um relatório de análise estatística que permitirá:

- Segmentação Aprimorada: Criação de pelo menos dois segmentos de clientes (ex: "Clientes de Alto Valor" e "Visitantes Engajados sem Compra") para direcionamento de campanhas de marketing personalizadas.

- Otimização de Marketing: Direcionar o orçamento de marketing para ações focadas nos comportamentos que mais se correlacionam com compras de alto valor, aumentando o Retorno Sobre o Investimento (ROI).

- Melhoria da Experiência do Usuário (UX): Fornecer à equipe de produto dados que possam justificar testes A/B ou melhorias em áreas do site frequentadas por usuários que não convertem.

## 2. Importação das Bibliotecas

In [1]:
# Importando a biblioteca NumPy
import numpy as np

In [2]:
# Outros Imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Definir uma semente para reprodutibilidade dos resultados
np.random.seed(42)

In [5]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [6]:
%watermark --iversions

matplotlib: 3.10.8
numpy     : 2.4.2
pandas    : 3.0.0
platform  : 1.0.9
seaborn   : 0.13.2



## 3. Função Para Geração de Dados Fictícios

Vamos criar um conjunto de dados para 500 usuários. Cada usuário terá 4 métricas associadas descritas abaixo.

- visitas: Número de vezes que o usuário visitou o site no mês.

- tempo_no_site: Tempo total em minutos que o usuário passou no site.

- itens_no_carrinho: Número de itens que o usuário adicionou ao carrinho.

- valor_compra: O valor total em R$ da compra realizada pelo usuário no mês.

In [7]:
# Definir o número de usuários
num_usuarios = 500

In [8]:
# 1. Gerar o número de visitas (entre 1 e 50)
visitas = np.random.randint(1, 51, size=num_usuarios)

A distribuição normal é um tipo de distribuição de probabilidade contínua, simétrica em torno da média, com formato característico de sino. Nesse tipo de distribuição os valores próximos da média são mais frequentes e a probabilidade de observar valores extremos diminui gradualmente conforme se afastam do centro. É definida por dois parâmetros: a média (que indica o centro da distribuição) e o desvio padrão (que indica a dispersão dos dados). É muito usada em Estatística e Ciência de Dados porque muitos fenômenos naturais e sociais tendem a seguir esse padrão.

In [ ]:
# 2. Gerar o tempo no site (distribuição normal, correlacionado com as visitas)
# Média de 20 min, desvio padrão de 5, com um bônus por visita
tempo_no_site = np.random.normal(loc=20, scale=5, size=num_usuarios) + (visitas * 0.5)
tempo_no_site = np.round(tempo_no_site, 2)  # Arredondar para 2 casas decimais

In [ ]:
# 3. Gerar o número de itens no carrinho (dependente das visitas e do tempo)
# Usuários que visitam mais e passam mais tempo, tendem a adicionar mais itens
itens_no_carrinho = np.random.randint(0, 8, size=num_usuarios) + (visitas // 10)

In [ ]:
# Garante que o tempo no site também influencie positivamente
itens_no_carrinho = (itens_no_carrinho + (tempo_no_site // 15)).astype(int)

In [ ]:
# 4. Gerar o valor da compra (correlacionado com os itens no carrinho)
# Preço médio por item de R$ 35, com alguma variação aleatória
valor_compra = (itens_no_carrinho * 35) + np.random.normal(
    loc=0, scale=10, size=num_usuarios
)

In [ ]:
type(valor_compra)

In [ ]:
type(itens_no_carrinho)

In [ ]:
# Se não houver itens no carrinho, o valor da compra deve ser 0
valor_compra[itens_no_carrinho == 0] = 0
valor_compra[valor_compra < 0] = 0 # Corrigir valores negativos que possam surgir
valor_compra = np.round(valor_compra, 2)